In [2]:
import pandas as pd
import os
from pathlib import Path
# import sys, traceback
import subprocess
import json
# import re
# import Bio.PDB

<span style="Times New Roman; font-size:2em;">**Loading data**</span>

In [3]:
curated_csv_data = []
with open('curated_data.csv', 'r') as csv_file:
    for line in csv_file:
        row = line.replace('\n','').split(',')
        curated_csv_data.append(row[:12])

curated_df = pd.DataFrame(curated_csv_data[1:], columns = ['M-CSA ID',
 'Uniprot IDs',
 'PDB ID',
 'EC',
 'residue/reactant/product/cofactor',
 'RESIDUE TYPE',
 'CHAIN ID',
 'RESIDUE NUMBER',
 'function location/name',
 'ROLE',
 'ROLE_TYPE',
 'PARENT ROLE'])

curated_df['RESIDUE NUMBER'] = curated_df['RESIDUE NUMBER'].astype('string')
curated_df['RESIDUE TYPE'] = curated_df['RESIDUE TYPE'].str.upper()
curated_df

,M-CSA ID,Uniprot IDs,PDB ID,EC,residue/reactant/product/cofactor,RESIDUE TYPE,CHAIN ID,RESIDUE NUMBER,function location/name,ROLE,ROLE_TYPE,PARENT ROLE
0,M0001,P56868,1b73,5.1.1.3,residue,ASP,A,7,side_chain,activator,spectator,activator
1,M0001,P56868,1b73,5.1.1.3,residue,ASP,A,7,side_chain,hydrogen bond acceptor,interaction,None
2,M0001,P56868,1b73,5.1.1.3,residue,ASP,A,7,side_chain,proton acceptor,reactant,proton shuttle (general acid/base)
3,M0001,P56868,1b73,5.1.1.3,residue,ASP,A,7,side_chain,electrostatic stabiliser,spectator,electrostatic interaction
4,M0001,P56868,1b73,5.1.1.3,residue,ASP,A,7,side_chain,hydrogen bond donor,interaction,None
...,...,...,...,...,...,...,...,...,...,...,...,...
28183,M0966,P15723,4x9e,3.1.5.1,reactant,,,61429,dGTP(4-),,,
28184,M0966,P15723,4x9e,3.1.5.1,reactant,,C00001,15377,water,,,
28185,M0966,P15723,4x9e,3.1.5.1,product,,C00330,17172,2'-deoxyguanosine,,,
28186,M0966,P15723,4x9e,3.1.5.1,product,,C00536,18036,triphosphate(5-),,,


In [0]:
residues_roles_df = pd.read_csv('literature_pdb_residues_roles.csv')
residues_roles_df['RESIDUE NUMBER'] = residues_roles_df['RESIDUE NUMBER'].astype('string')
residues_roles_df['RESIDUE TYPE'] = residues_roles_df['RESIDUE TYPE'].str.upper()
# residues_roles_df.dtypes

In [0]:
# Dataframe for storing the results
results_df = pd.read_csv('results.csv')
results_df['RESIDUE NUMBER'] = results_df['RESIDUE NUMBER'].astype('string')
# results_df.dtypes

In [0]:
# Dataframe for storing the results
enzymes_without_cofactors_df = pd.read_csv('enzymes_without_cofactors.csv')
enzymes_without_cofactors_df['RESIDUE NUMBER'] = enzymes_without_cofactors_df['RESIDUE NUMBER'].astype('string')
# enzymes_without_cofactors_df.dtypes

<span style="Times New Roman; font-size:2em;">**Merging Data**</span>

In [7]:
df_merged = pd.merge(results_df, residues_roles_df, how = 'left', on = ['PDB ID', 'RESIDUE TYPE', 'CHAIN ID', 'RESIDUE NUMBER'])
df_merged.to_csv( 'merged.csv', index=False)
df_merged

,PDB ID,RESIDUE TYPE,RESIDUE NUMBER,CHAIN ID,pKa,model-pKa,delta-pKa,ligand atom-type,SITE NUMBER,CHEMICAL FUNCTION,EVIDENCE TYPE,LITERATURE ENTRY,ROLE_TYPE,ROLE,PARENT ROLE
0,12as,ASP,42,A,5.05,3.8,1.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,12as,ASP,46,A,2.73,3.8,-1.07,NaN,0.0,S,LIT,12as,interaction,hydrogen bond acceptor,None
2,12as,ASP,46,A,2.73,3.8,-1.07,NaN,0.0,S,LIT,12as,interaction,hydrogen bond donor,None
3,12as,ASP,46,A,2.73,3.8,-1.07,NaN,0.0,S,LIT,12as,reactant,proton acceptor,proton shuttle (general acid/base)
4,12as,ASP,46,A,2.73,3.8,-1.07,NaN,0.0,S,LIT,12as,reactant,proton donor,proton shuttle (general acid/base)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239198,9pap,ARG,111,A,12.40,12.5,-0.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
239199,9pap,ARG,145,A,12.40,12.5,-0.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
239200,9pap,ARG,188,A,13.69,12.5,1.19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
239201,9pap,ARG,191,A,11.92,12.5,-0.58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
sum(df_merged.ROLE.notnull())

In [8]:
df_merged = pd.merge(enzymes_without_cofactors_df, residues_roles_df, how = 'left', on = ['PDB ID', 'RESIDUE TYPE', 'CHAIN ID', 'RESIDUE NUMBER'])
df_merged.to_csv( 'merged_enzymes_without_cofactors.csv', index=False)
df_merged

,PDB ID,RESIDUE TYPE,RESIDUE NUMBER,CHAIN ID,pKa,model-pKa,delta-pKa,ligand atom-type,SITE NUMBER,CHEMICAL FUNCTION,EVIDENCE TYPE,LITERATURE ENTRY,ROLE_TYPE,ROLE,PARENT ROLE
0,1a26,ASP,671,A,3.65,3.8,-0.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1a26,ASP,678,A,3.56,3.8,-0.24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1a26,ASP,692,A,3.69,3.8,-0.11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1a26,ASP,722,A,4.04,3.8,0.24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1a26,ASP,731,A,3.53,3.8,-0.27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87992,9pap,ARG,111,A,12.40,12.5,-0.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87993,9pap,ARG,145,A,12.40,12.5,-0.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87994,9pap,ARG,188,A,13.69,12.5,1.19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87995,9pap,ARG,191,A,11.92,12.5,-0.58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df_merged = pd.merge(enzymes_without_cofactors_df, curated_df, how = 'left', on = ['PDB ID', 'RESIDUE TYPE', 'CHAIN ID', 'RESIDUE NUMBER'])
df_merged.to_csv( 'merged_enzymes_without_cofactors_curated.csv', index=False)
df_merged

,PDB ID,RESIDUE TYPE,RESIDUE NUMBER,CHAIN ID,pKa,model-pKa,delta-pKa,ligand atom-type,M-CSA ID,Uniprot IDs,EC,residue/reactant/product/cofactor,function location/name,ROLE,ROLE_TYPE,PARENT ROLE
0,1a26,ASP,671,A,3.65,3.8,-0.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1a26,ASP,678,A,3.56,3.8,-0.24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1a26,ASP,692,A,3.69,3.8,-0.11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1a26,ASP,722,A,4.04,3.8,0.24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1a26,ASP,731,A,3.53,3.8,-0.27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89703,9pap,ARG,111,A,12.40,12.5,-0.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89704,9pap,ARG,145,A,12.40,12.5,-0.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89705,9pap,ARG,188,A,13.69,12.5,1.19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89706,9pap,ARG,191,A,11.92,12.5,-0.58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
